In [14]:
import requests
import json
import os
import cv2
from PIL import Image
import shutil
import pandas as pd
import random
import numpy as np

path = os.getcwd()
data = ""
names = []

#### This assumes that you have a dataset with the following structure
dataset

|--> train
<br>
    |--> images
    <br>
    |--> labels

|--> valid
<br>
    |--> images
    <br>
    |--> labels

|--> test
<br>
    |--> images
    <br>
    |--> labels

data.yaml

#### 1. rename all files in the dataset to consistent names

#### 2. split samples based on how many objects there are per image (try to approximate as well as possible)

#### 3. create data.yaml file


# Functions

In [10]:
def delete_all_folders():
    shutil.rmtree(path + "/combined_ds")
    shutil.rmtree(path + "/org_ds")
    shutil.rmtree(path + "/final_ds")

# roboflow does a weird thing where it inserts .rf in the middle so you need to remove it
def clean_names(dataset_name):
    for split in os.listdir(dataset_name):
        if split == 'test' or split == 'train' or split == 'valid':
            print(split)
            for img in os.listdir(f'{dataset_name}/{split}/images'):
                # rename image at this path to remove "_jpg"
                name, extension = os.path.splitext(img)
                # assume that the file is already in jpg form
                name = name[:name.rfind('_')]
                os.rename(f'{dataset_name}/{split}/images/{img}', f'{dataset_name}/{split}/images/{name}{extension}')

    
def set_name_and_paths(dataset_name):
    # read data.yaml from b_ds
    global data
    global names
    with open(f'{dataset_name}/data.yaml', 'r') as f:
        data = f.read()

    # remove everything after "roboflow"
    
    data = data.split('roboflow')[0] # this will be the same...

    # get all names from data.yaml
    names = data.split('names: ')[1]

    # convert string names to list
    names = eval(names)
    print(names)

    # rewrite train, val, test to go to final_ds/{split}/images instead of ../{split}/images
    data = data.replace('../train/images', 'final_ds/train/images')
    data = data.replace('../valid/images', 'final_ds/valid/images')
    data = data.replace('../test/images', 'final_ds/test/images')

    print(data)


def organize_to_names():
    # assert background_proportion >= 0 and background_proportion <= 1, "background_proportion must be between 0 and 1"

    names_with_freqs = [0 for i in range(len(names))]
    x=0

    for name in names:
        if os.path.exists(path + f"/org_ds/{name}"):
            shutil.rmtree(path + f"/org_ds/{name}")
        os.makedirs(path + f"/org_ds/{name}")
    
    if os.path.exists(path + f"/org_ds/labels"):
            shutil.rmtree(path + f"/org_ds/labels")
    os.makedirs(path + f"/org_ds/labels")

    if os.path.exists(path + f"/org_ds/backgrounds"):
        shutil.rmtree(path + f"/org_ds/backgrounds")
    os.makedirs(path + f"/org_ds/backgrounds")
    
    for label in os.listdir(path + "/combined_ds/labels"):
        # organize everything into folders based on
        
        with open(path + f"/combined_ds/labels/{label}") as file:
            # read first line
            asdf = file.readline().split(" ")[0]
            # print(asdf)
            try:
                numval = int(asdf)
                real_name = names[numval]
                
                # move image and label to folder
                names_with_freqs[numval] += 1

                name, extension = os.path.splitext(real_name)

                # fix fix fix 
                shutil.copy(path + f"/combined_ds/images/{name}{extension}", path + f"/org_ds/{real_name}/{real_name}.jpg")
                shutil.copy(path + f"/combined_ds/labels/{label}", path + f"/org_ds/labels/{real_name}.txt")
                    
                x+=1
            except:
                # label does not exist
                shutil.copy(path + f"/combined_ds/images/{label[:-4]}.jpg", path + f"/org_ds/backgrounds/{label[:-4]}_background.jpg")
                continue


def get_splits(ratios, background_proportion=0.1):
    ftrain, fval, ftest = np.array([]), np.array([]), np.array([])
    # stratify splitting of data
    print("getting splits")
    for name in names:
        allFileNames = os.listdir(path + f"/org_ds/{name}")
        np.random.seed(42)
        np.random.shuffle(allFileNames)

        train, val, test = np.split(np.array(allFileNames),[int(len(allFileNames)*0.8), int(len(allFileNames)*0.9)])

        ftrain = np.concatenate((ftrain, train))
        fval = np.concatenate((fval, val))
        ftest = np.concatenate((ftest, test))

        print(name, len(train), len(val), len(test))
    
    

    if background_proportion > 0:
        # calculate number of backgrounds to add
        num_backgrounds = (len(ftrain)*background_proportion)/(1-background_proportion)
        allFileNames = os.listdir(path + f"/org_ds/backgrounds")
        np.random.seed(42)
        np.random.shuffle(allFileNames)

        train, val, test = np.split(np.array(allFileNames),[int(len(allFileNames)*0.8), int(len(allFileNames)*0.9)])

        train = train[:num_backgrounds]
        val = val[:num_backgrounds]
        test = test[:num_backgrounds]

        ftrain = np.concatenate((ftrain, train))
        fval = np.concatenate((fval, val))
        ftest = np.concatenate((ftest, test))

    print("final lengths after stratified split: ", len(ftrain), len(fval), len(ftest))
    
    global globftrain
    global globfval
    global globftest
    globftrain = ftrain
    globfval = fval
    globftest = ftest
    
    return ftrain, fval, ftest

def generate_dataset(dataset_name, exclude_classes = ['anabaena'], ratios = [0.8, 0.1, 0.1], background_proportion=0.1):
    clean_names(dataset_name)
    set_name_and_paths(dataset_name)
    organize_to_names(background_proportion)
    ftrain, fval, ftest = get_splits(ratios, background_proportion)


In [11]:
# delete_all_folders()
clean_names('dataset')
set_name_and_paths('dataset')

valid
test
train


## This program works by 

